# LLMs + mockstack

A few simple examples for using `mockstack` to mock various components in typical LLM-driven use cases.

In [7]:
# Install langchain dependencies with the following or using `uv` depending on your venv setup:

#!pip install -q langchain langchain-openai
# or:
#!uv pip install langchain langchain-openai

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

## Example #1: Static template-based mocking (`filefixtures` strategy)

Here we simply use the **filefixtures** strategy to route requesets coming in for a certain URL to a template file with the appropriate name.

For the below example you'll want to make sure:

- mockstack is running at http://localhost:8000 which are the default settings
- `MOCKSTACK__TEMPLATES_DIR` is pointing to a valid directory with a template file called `openai-v1-chat-completions.j2`. See the [included file](./templates/openai-v1-chat-completions.j2) with same name in the ./templates sub-directory of this example for a a template with a valid response based on [OpenAI API reference](https://platform.openai.com/docs/api-reference/chat/create).
- the `MOCKSTACK__FILEFIXTURES_ENABLE_TEMPLATES_FOR_POST` flag is set to true (which it should be by default)


If everything is setup correctly, you should get back the mocked response in the correct format from the template and the below assert should pass.

In [2]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    base_url="http://localhost:8000/openai/v1",
    api_key="SOME_STRING_THAT_DOES_NOT_MATTER",
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)

assert ai_msg.content == "Hello! How can I assist you today?"

Below is an identical example, but using OpenAI via Microsoft Azure.

The only difference is that under the hood `LangChain` will hit an API URL of the form `/openai/v1/deployments/gpt-4o/chat/completions?api-version=<api-version>`.

We can again handle this easily using templates by just creating a file with the name `openai-v1-deployments-gpt-4o-chat-completions.j2`.

Remember you can also create conditional logic inside that file using Jinja syntax to return different mock responses depending on the request parameters, for instance using the query parameter `api-version`.

In [3]:
from langchain_openai import AzureChatOpenAI


llm = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2024-02-15-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    base_url="http://localhost:8000/openai/v1",
    api_key="SOME_STRING_THAT_DOES_NOT_MATTER",
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)

assert ai_msg.content == "Hello! How can I assist you today?"

## Example #2: Dynamic template-based mocking with ollama integration

In this example we use the [ollama-python](https://github.com/ollama/ollama-python) integration to return mock responses that actually come from a real LLM running on your host! This is a great way to go a step further in development, debugging, and integration testing scenarios where LLMs and their non-determinism are important to capture.

For this example you will need to make sure you have the following:

* mockstack installed with the optional `llm` dependencies:

    ```bash
    uv pip install mockstack[llm]
    ```

* [ollama](https://ollama.com/) installed locally with a particular model you'd like to use.

In [4]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    base_url="http://localhost:8000/ollama/openai/v1",
    api_key="SOME_STRING_THAT_DOES_NOT_MATTER",
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "mockstack is pretty cool. But LLMs are way cooler"),
]
ai_msg = llm.invoke(messages)

# This should be "Je aime le programmation." or similar, but subject to some stochasticity based on LLM.
print(ai_msg.content)

InternalServerError: Internal Server Error

## Example #3: mocking out a tool call

** COMING SOON **